# Working Notebook

__Phase 3 Project - Chicago Traffic Crash Classification__

### Business Understanding

### Data Understanding and Preparation

Load dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer


from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV


Import data files

In [2]:
crashes = pd.read_csv('C:/Users/angie/Desktop/group_project/Chicago-Crashes/data/Traffic_Crashes_-_Crashes.csv', low_memory=False)
people = pd.read_csv('C:/Users/angie/Desktop/group_project/Chicago-Crashes/data/Traffic_Crashes_-_People.csv', low_memory=False)
vehicles = pd.read_csv('C:/Users/angie/Desktop/group_project/Chicago-Crashes/data/Traffic_Crashes_-_Vehicles.csv', low_memory = False)

In [3]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE_EST_I               41029 non-null   object 
 3   CRASH_DATE                     541142 non-null  object 
 4   POSTED_SPEED_LIMIT             541142 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 6   DEVICE_CONDITION               541142 non-null  object 
 7   WEATHER_CONDITION              541142 non-null  object 
 8   LIGHTING_CONDITION             541142 non-null  object 
 9   FIRST_CRASH_TYPE               541142 non-null  object 
 10  TRAFFICWAY_TYPE                541142 non-null  object 
 11  LANE_CNT                       198968 non-null  float64
 12  ALIGNMENT                     

In [4]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

Remove columns with 80% or more of null values.

In [5]:
df_list=[crashes, people, vehicles]
for df in df_list:
    min_count =  int((20/100)*df.shape[0] + 1)
    df = df.dropna(axis=1, thresh=min_count, inplace=True)
    

In [6]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   RD_NO                1097786 non-null  object 
 3   CRASH_DATE           1107267 non-null  object 
 4   UNIT_NO              1107267 non-null  int64  
 5   UNIT_TYPE            1105670 non-null  object 
 6   VEHICLE_ID           1081934 non-null  float64
 7   MAKE                 1081929 non-null  object 
 8   MODEL                1081786 non-null  object 
 9   LIC_PLATE_STATE      987487 non-null   object 
 10  VEHICLE_YEAR         905696 non-null   float64
 11  VEHICLE_DEFECT       1081934 non-null  object 
 12  VEHICLE_TYPE         1081934 non-null  object 
 13  VEHICLE_USE          1081934 non-null  object 
 14  TRAVEL_DIRECTION     1081934 non-null  object 
 15

Drop columns that will not be used

In [7]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   RD_NO                1097786 non-null  object 
 3   CRASH_DATE           1107267 non-null  object 
 4   UNIT_NO              1107267 non-null  int64  
 5   UNIT_TYPE            1105670 non-null  object 
 6   VEHICLE_ID           1081934 non-null  float64
 7   MAKE                 1081929 non-null  object 
 8   MODEL                1081786 non-null  object 
 9   LIC_PLATE_STATE      987487 non-null   object 
 10  VEHICLE_YEAR         905696 non-null   float64
 11  VEHICLE_DEFECT       1081934 non-null  object 
 12  VEHICLE_TYPE         1081934 non-null  object 
 13  VEHICLE_USE          1081934 non-null  object 
 14  TRAVEL_DIRECTION     1081934 non-null  object 
 15

In [8]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE                     541142 non-null  object 
 3   POSTED_SPEED_LIMIT             541142 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 5   DEVICE_CONDITION               541142 non-null  object 
 6   WEATHER_CONDITION              541142 non-null  object 
 7   LIGHTING_CONDITION             541142 non-null  object 
 8   FIRST_CRASH_TYPE               541142 non-null  object 
 9   TRAFFICWAY_TYPE                541142 non-null  object 
 10  LANE_CNT                       198968 non-null  float64
 11  ALIGNMENT                      541142 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [9]:
crashes_mod = crashes.drop(['CRASH_DATE','RD_NO','REPORT_TYPE', 'DATE_POLICE_NOTIFIED', 'STREET_NO', 
              'STREET_DIRECTION', 'STREET_NAME', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
             'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 
             'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN'], axis =1)

In [10]:
people_mod = people.drop(['CITY', 'ZIPCODE','RD_NO'], axis =1)

In [11]:
vehicles_mod = vehicles.drop(['RD_NO', 'CRASH_DATE', 'UNIT_NO', 'AREA_01_I'], axis =1)

In [12]:
people_mod.shape

(1195747, 19)

In [13]:
vehicles_mod.shape

(1107267, 15)

In [14]:
crashes_mod.shape

(541142, 26)

## First Simple Model

In [15]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE                     541142 non-null  object 
 3   POSTED_SPEED_LIMIT             541142 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 5   DEVICE_CONDITION               541142 non-null  object 
 6   WEATHER_CONDITION              541142 non-null  object 
 7   LIGHTING_CONDITION             541142 non-null  object 
 8   FIRST_CRASH_TYPE               541142 non-null  object 
 9   TRAFFICWAY_TYPE                541142 non-null  object 
 10  LANE_CNT                       198968 non-null  float64
 11  ALIGNMENT                      541142 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [16]:
crashes.MOST_SEVERE_INJURY.value_counts()

NO INDICATION OF INJURY     468994
NONINCAPACITATING INJURY     39675
REPORTED, NOT EVIDENT        21763
INCAPACITATING INJURY         9040
FATAL                          547
Name: MOST_SEVERE_INJURY, dtype: int64

Set up target variable:
 - 0: NO INDICATION OF INJURY, NONINCAPACITATING INJURY, REPORTED, NOT EVIDENT
 - 1: INCAPACITATING INJURY, FATAL
 

In [17]:
crashes['TARGET']= crashes['MOST_SEVERE_INJURY'].map({'NO INDICATION OF INJURY': 0,
                                                      'NONINCAPACITATING INJURY': 0,
                                                      'REPORTED, NOT EVIDENT': 0,
                                                      'INCAPACITATING INJURY': 1,
                                                      'FATAL': 1})

In [18]:
crashes.TARGET.fillna(0, inplace=True)

In [19]:
crashes.TARGET.value_counts()

0.0    531555
1.0      9587
Name: TARGET, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X = crashes.drop(['MOST_SEVERE_INJURY','INJURIES_TOTAL', 'INJURIES_FATAL','INJURIES_INCAPACITATING',
                      'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
                      'INJURIES_NO_INDICATION', 'TARGET', 'CRASH_RECORD_ID', 'CRASH_DATE', 'LATITUDE', 
                      'LONGITUDE','LOCATION'], axis=1)
y = crashes['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
X_train_nums = X_train.select_dtypes(include=['float64', 'int64'])
X_train_cat = X_train.select_dtypes('object')

In [22]:
numerical_pipeline = Pipeline(steps=[
    ('knn_impute', KNNImputer()),
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.columns),
    ('categorical', categorical_pipeline, X_train_cat.columns)
])

In [23]:
model_pipe = Pipeline(steps=[
    ('trans', trans),
    ('dc', DummyClassifier(strategy='most_frequent'))
])

In [24]:
#model_pipe.fit(X_train, y_train)

In [25]:
#model_pipe.score(X_train, y_train)

In [26]:
#model_pipe.score(X_test, y_test)

### Merging Dataframes

From the people data, the target will be the injury classification so that the model will predict the severity of injuries.

In [27]:
people_mod.INJURY_CLASSIFICATION.value_counts()

NO INDICATION OF INJURY     1096283
NONINCAPACITATING INJURY      55296
REPORTED, NOT EVIDENT         32092
INCAPACITATING INJURY         10827
FATAL                           656
Name: INJURY_CLASSIFICATION, dtype: int64

To avoid too many repeated rows from crashes_dropped_df, we'll use the `CRASH_RECORD_ID` to only merge in rows from people_dropped_df where `DRIVER_TYPE == DRIVER`, then use the `VEHICLE_ID` to only merge in that driver's vehicle from vehicles_dropped_df.

This will allow us to still retain over 75% of the rows from people.

In [28]:
people_mod.PERSON_TYPE.value_counts(normalize=True)

DRIVER                 0.776163
PASSENGER              0.204485
PEDESTRIAN             0.011393
BICYCLE                0.006911
NON-MOTOR VEHICLE      0.000866
NON-CONTACT VEHICLE    0.000181
Name: PERSON_TYPE, dtype: float64

In [29]:
people_crashes_df = people_mod.merge(crashes_mod, on='CRASH_RECORD_ID', how='left')

In [30]:
people_crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195747 entries, 0 to 1195746
Data columns (total 44 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   PERSON_ID                1195747 non-null  object 
 1   PERSON_TYPE              1195747 non-null  object 
 2   CRASH_RECORD_ID          1195747 non-null  object 
 3   VEHICLE_ID               1172146 non-null  float64
 4   CRASH_DATE               1195747 non-null  object 
 5   SEAT_NO                  244512 non-null   float64
 6   STATE                    890189 non-null   object 
 7   SEX                      1177665 non-null  object 
 8   AGE                      852450 non-null   float64
 9   DRIVERS_LICENSE_STATE    705542 non-null   object 
 10  DRIVERS_LICENSE_CLASS    608027 non-null   object 
 11  SAFETY_EQUIPMENT         1192258 non-null  object 
 12  AIRBAG_DEPLOYED          1173141 non-null  object 
 13  EJECTION                 1181206 non-null 

In [31]:
people_crashes_df.INJURY_CLASSIFICATION.value_counts()

NO INDICATION OF INJURY     1096283
NONINCAPACITATING INJURY      55296
REPORTED, NOT EVIDENT         32092
INCAPACITATING INJURY         10827
FATAL                           656
Name: INJURY_CLASSIFICATION, dtype: int64

In [32]:
people_crashes_df['TARGET'] = people_crashes_df['INJURY_CLASSIFICATION'].map({np.nan: 0,
                                                      'NO INDICATION OF INJURY': 0,
                                                      'NONINCAPACITATING INJURY': 1,
                                                      'REPORTED, NOT EVIDENT': 1,
                                                      'INCAPACITATING INJURY': 2,
                                                      'FATAL': 3})

In [33]:
people_crashes_df.TARGET.isna().sum()

0

In [34]:
people_crashes_df['CRASH_YEAR']=pd.to_datetime(people_crashes_df['CRASH_DATE']).dt.year

In [35]:
people_crashes_df['CRASH_YEAR'].value_counts()

2018    265694
2019    263972
2020    202084
2017    185328
2021    161704
2016     96020
2015     20931
2014        11
2013         3
Name: CRASH_YEAR, dtype: int64

In [36]:
people_crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195747 entries, 0 to 1195746
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   PERSON_ID                1195747 non-null  object 
 1   PERSON_TYPE              1195747 non-null  object 
 2   CRASH_RECORD_ID          1195747 non-null  object 
 3   VEHICLE_ID               1172146 non-null  float64
 4   CRASH_DATE               1195747 non-null  object 
 5   SEAT_NO                  244512 non-null   float64
 6   STATE                    890189 non-null   object 
 7   SEX                      1177665 non-null  object 
 8   AGE                      852450 non-null   float64
 9   DRIVERS_LICENSE_STATE    705542 non-null   object 
 10  DRIVERS_LICENSE_CLASS    608027 non-null   object 
 11  SAFETY_EQUIPMENT         1192258 non-null  object 
 12  AIRBAG_DEPLOYED          1173141 non-null  object 
 13  EJECTION                 1181206 non-null 

In [37]:
from sklearn.model_selection import train_test_split
X = people_crashes_df.drop(['PERSON_ID', 'CRASH_RECORD_ID','VEHICLE_ID','CRASH_DATE', 'LATITUDE', 
                      'LONGITUDE','LOCATION', 'TARGET', 'INJURY_CLASSIFICATION'], axis=1)
y = people_crashes_df['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [38]:
X_train.head()

,PERSON_TYPE,SEAT_NO,STATE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,...,HIT_AND_RUN_I,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,BEAT_OF_OCCURRENCE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_YEAR
635474,DRIVER,NaN,IL,F,34.0,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"OVER $1,500",IMPROPER TURNING/NO SIGNAL,FAILING TO YIELD RIGHT-OF-WAY,1922.0,2.0,11,7,2,2019
425369,DRIVER,NaN,IL,M,29.0,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"$501 - $1,500",DISREGARDING TRAFFIC SIGNALS,NOT APPLICABLE,1232.0,2.0,2,2,3,2018
1152397,DRIVER,NaN,NaN,M,39.0,IL,NaN,SAFETY BELT USED,NOT APPLICABLE,NONE,...,NaN,"$501 - $1,500",NOT APPLICABLE,NOT APPLICABLE,823.0,2.0,17,3,5,2021
75101,DRIVER,NaN,IL,M,26.0,WA,9,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"OVER $1,500",DISREGARDING ROAD MARKINGS,NOT APPLICABLE,1732.0,2.0,23,1,4,2018
265075,DRIVER,NaN,IL,M,24.0,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,...,NaN,"$501 - $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,FOLLOWING TOO CLOSELY,831.0,2.0,6,1,6,2017


In [40]:
#XX = ['STATE', 'DRIVERS_LICENSE_STATE']
#UNKNOWN = ['EJECTION', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION']
#USAGE_UNKNOWN = ['SAFETY_EQUIPMENT']
#DEPLOYMENT_UNKNOWN = ['AIRBAG_DEPLOYED']

In [41]:
from sklearn.preprocessing import FunctionTransformer

def seat_no_transformer(df):
    '''
    Set all SEAT_NO = 1 if PERSON_TYPE = 'DRIVER'
    and fill the rest with 0.
    '''
    df.loc[df.PERSON_TYPE == 'DRIVER']['SEAT_NO'] = 1
    df['SEAT_NO'].fillna(value=0, inplace=True)
    return df

In [42]:
def speed_transformer(df):
    df['POSTED_SPEED_LIMIT'][df.POSTED_SPEED_LIMIT == 0] = np.nan
    df['POSTED_SPEED_LIMIT'][(df.POSTED_SPEED_LIMIT %5) != 0] = np.nan
    return df

In [43]:
def beat_transformer(df):
    df['BEAT_OF_OCCURENCE'] = df['BEAT_OF_OCCURENCE'].astype(str)
    df['BEAT_OF_OCCURENCE'] = df['BEAT_OF_OCCURENCE'].apply(lambda x: x[:-4])
    return df

In [44]:
def age_transformer(df):
    df['AGE'][df.AGE < 0] = np.nan
    df.loc[(df.AGE == 0) & (df.PERSON_TYPE == 'DRIVER')]['AGE'] = np.nan
    return df

In [45]:
def fill_categories(df):
    df.fillna({'STATE': 'XX',
              'DRIVERS_LICENSE_STATE': 'UNKNOWN',
              'DRIVERS_LICENSE_CLASS': 'UNKNOWN',
              'EJECTION': 'UNKNOWN',
              'DRIVER_ACTION': 'UNKNOWN',
              'DRIVER_VISION': 'UNKNOWN',
              'PHYSICAL_CONDITION': 'UNKNOWN',
              'SAFETY_EQUIPMENT': 'USAGE_UNKNOWN',
              'AIRBAG_DEPLOYED': 'DEPLOYMENT_UNKNOWN',
              'SEX': 'UNKNOWN',
              'HIT_AND_RUN_I': 'N',
              'INTERSECTION_RELATED_I': 'N',
              'BAC_RESULT': 'TEST NOT OFFERED'
              }, inplace = True)
    return df

In [48]:
cleaning_pipeline = Pipeline(steps = [
    ('age_trans', FunctionTransformer(age_transformer)),
    ('seat_no', FunctionTransformer(seat_no_transformer)),
    ('speed_trans', FunctionTransformer(speed_transformer)),
    ('beat_trans', FunctionTransformer(beat_transformer)),
    ('fill_cat', FunctionTransformer(fill_categories))
])

In [50]:
X_train.head()

,PERSON_TYPE,SEAT_NO,STATE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,...,HIT_AND_RUN_I,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,BEAT_OF_OCCURRENCE,NUM_UNITS,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_YEAR
635474,DRIVER,0.0,IL,F,34.0,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"OVER $1,500",IMPROPER TURNING/NO SIGNAL,FAILING TO YIELD RIGHT-OF-WAY,1922.0,2.0,11,7,2,2019
425369,DRIVER,0.0,IL,M,29.0,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"$501 - $1,500",DISREGARDING TRAFFIC SIGNALS,NOT APPLICABLE,1232.0,2.0,2,2,3,2018
1152397,DRIVER,0.0,NaN,M,39.0,IL,NaN,SAFETY BELT USED,NOT APPLICABLE,NONE,...,NaN,"$501 - $1,500",NOT APPLICABLE,NOT APPLICABLE,823.0,2.0,17,3,5,2021
75101,DRIVER,0.0,IL,M,26.0,WA,9,SAFETY BELT USED,DID NOT DEPLOY,NONE,...,NaN,"OVER $1,500",DISREGARDING ROAD MARKINGS,NOT APPLICABLE,1732.0,2.0,23,1,4,2018
265075,DRIVER,0.0,IL,M,24.0,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,...,NaN,"$501 - $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,FOLLOWING TOO CLOSELY,831.0,2.0,6,1,6,2017


In [49]:
X_train_transformed = cleaning_pipeline.fit_transform(X_train)

<ipython-input-44-5a87e6e6625e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AGE'][df.AGE < 0] = np.nan
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\pandas\core\series.py:1020: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(~key, value, inplace=True)
<ipython-input-44-5a87e6e6625e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(df.AGE == 

KeyError: 'BEAT_OF_OCCURENCE'

In [ ]:
X_train_nums = X_train.select_dtypes(include = ['float64', 'int64'])
X_train_cat = X_train.select_dtypes('object')

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer


In [ ]:
numerical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(missing_values = np.nan, strategy = 'median')),
    ('ss', StandardScaler())
])
                
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first',
                         sparse=False))
])

trans = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, X_train_nums.select_dtypes(include = ['float64', 'int64']).columns),
    ('categorical', categorical_pipeline, X_train.select_dtypes('object').columns)
])

In [ ]:
model_pipe = Pipeline(steps = [
    ('trans', trans),
    ('dc', DummyClassifier(strategy = 'most_frequent'))
])

In [ ]:
model_pipe.fit(X_train_transformed, y_train)